In [105]:
import pandas as pd
import numpy as np

import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
#from transformers import Trainer, TrainingArguments
from transformers import EvalPrediction
from torch.utils.data import DataLoader, Dataset
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam

from sklearn.metrics import f1_score

## TASK 1 - CORPUS

In [106]:
def load_and_merge_data():
    
    #encodng the data into pandas.DataFrame objects
    url_a_test = 'Data/arguments-test.tsv'
    df_a_test = pd.read_csv(url_a_test, sep='\t')

    url_a_training = 'Data/arguments-training.tsv'
    df_a_training = pd.read_csv(url_a_training, sep='\t')

    url_a_validation = 'Data/arguments-validation.tsv'
    df_a_validation = pd.read_csv(url_a_validation, sep='\t')

    url_l_test = 'Data/labels-test.tsv'
    df_l_test = pd.read_csv(url_l_test, sep='\t')

    url_l_training = 'Data/labels-training.tsv'
    df_l_training = pd.read_csv(url_l_training, sep='\t')

    url_l_validation = 'Data/labels-validation.tsv'
    df_l_validation = pd.read_csv(url_l_validation, sep='\t')

    #merge argument dataframes with label dataframes
    df_test = pd.merge(df_a_test, df_l_test, on='Argument ID')
    df_training = pd.merge(df_a_training, df_l_training, on='Argument ID')
    df_validation = pd.merge(df_a_validation, df_l_validation, on='Argument ID')

    return df_test, df_training, df_validation

In [107]:
df_test, df_training, df_validation = load_and_merge_data()

In [108]:
def merge_and_drop_columns(df):
    # Merge level 2 annotations to level 3 categories
    df['Openess to change'] = df[['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism']].any(axis=1).astype(int)
    df['Self-enhancement'] = df[['Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face']].any(axis=1).astype(int)
    df['Conservation'] = df[['Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility']].any(axis=1).astype(int)
    df['Self-transcendence'] = df[['Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity']].any(axis=1).astype(int)
    
    # Drop unuseful columns
    columns_to_drop = ['Argument ID', 'Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity']
    df = df.drop(columns=columns_to_drop)
    
    return df

df_test = merge_and_drop_columns(df_test)
df_training = merge_and_drop_columns(df_training)
df_validation = merge_and_drop_columns(df_validation)


In [125]:
df_test.head()

,Conclusion,Stance,Premise,Openess to change,Self-enhancement,Conservation,Self-transcendence
0,We should end affirmative action,0,affirmative action helps with employment equity.,0,1,1,1
1,We should end affirmative action,1,affirmative action can be considered discrimin...,0,1,0,1
2,We should ban naturopathy,1,naturopathy is very dangerous for the most vul...,0,1,1,1
3,We should prohibit women in combat,1,women shouldn't be in combat because they aren...,0,1,0,0
4,We should ban naturopathy,1,once eradicated illnesses are returning due to...,0,1,1,1


### DATA EXPLORATION
Ancora da inserire

### DATA PREPROCESSING

Encoding 'Stance' column into numerical format  

In [109]:
df_test['Stance'] = df_test['Stance'].replace({'in favor of': 1, 'against': 0}).astype(int)
df_training['Stance'] = df_training['Stance'].replace({'in favor of': 1, 'against': 0}).astype(int)
df_validation['Stance'] = df_validation['Stance'].replace({'in favor of': 1, 'against': 0}).astype(int)

Preparing data for tokenization input

In [110]:
labels_test = df_test.iloc[:, 3:7].values
labels_training = df_training.iloc[:, 3:7].values
labels_validation = df_validation.iloc[:, 3:7].values

stance_test = df_test['Stance'].values
stance_training = df_training['Stance'].values
stance_validation = df_validation['Stance'].values

Tokenization process and creation of a dataset structure compatible with the bert model 

In [126]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 100

class BertDatasetCreator(Dataset):
    def __init__(self, encodings, labels, tokenizer, max_length):
        self.encodings = encodings
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.encodings)
    
    def __getitem__(self, idx):
        item = str(self.encodings)
        item = ' '.join(item.split())
        
        encoded_dict = self.tokenizer.encode_plus(
            item,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        token_type_ids.append(encoded_dict['token_type_ids'])

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_masks, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(self.labels, dtype=torch.float)
        }

#### Applying the BertDatasetCreator and preparing the datasets for the three different type of BERT models

##### BERT w/C dataset

In [123]:
test_dataset_c = BertDatasetCreator(df_test['Conclusion'], labels_test, tokenizer, max_length)
train_dataset_c = BertDatasetCreator(df_training['Conclusion'], labels_training, tokenizer, max_length)
val_dataset_c = BertDatasetCreator(df_validation['Conclusion'], labels_validation, tokenizer, max_length)

DataLoader definition - which will supply the data to the neural network in batches for efficient training and processing

In [124]:
batch_size = 16
test_dataloader_c = DataLoader(test_dataset_c, batch_size=batch_size)
train_dataloader_c = DataLoader(train_dataset_c, batch_size=batch_size)
val_dataloader_c = DataLoader(val_dataset_c, batch_size=batch_size)

##### BERT w/CP

In [128]:
test_dataset_cp = BertDatasetCreator(df_test['Conclusion'] + ' ' + df_test['Premise'], labels_test, tokenizer, max_length)
train_dataset_cp = BertDatasetCreator(df_training['Conclusion'] + ' ' + df_training['Premise'], labels_training, tokenizer, max_length)
val_dataset_cp = BertDatasetCreator(df_validation['Conclusion'] + ' ' + df_validation['Premise'], labels_validation, tokenizer, max_length)

test_dataloader_cp = DataLoader(test_dataset_cp, batch_size=batch_size)
train_dataloader_cp = DataLoader(train_dataset_cp, batch_size=batch_size)
val_dataloader_cp = DataLoader(val_dataset_cp, batch_size=batch_size)

##### BERT w/CPS

## TASK 2 - MODEL DEFINITION

### BASELINE MODELS

Random uniform classifier

In [31]:
def create_random_uniform_classifier(category):
    """
    Creates a random classifier predicting 0 or 1 with uniform probability.
    inputs:
        category: Category to predict
    outputs: 
        a function that generates random predictions
    """
    def random_uniform_classifier(size):
        """
        Generates random uniform predictions for the given category.
        inputs: 
            size: number of predictions to generate
        outputs: 
            array of random uniform predictions
        """
        return np.random.choice([0, 1], size=size)
    
    return random_uniform_classifier

Majority classifier

In [32]:
def create_majority_classifier(category, majority_value):
    """
    Creates a majority classifier always predicting the most frequent valorization for the column.
    inputs:
        category: Category to predict
        majority_value: most frequent value (0 or 1)
    outputs:
        a function that generates majority predictions
    """
    def majority_classifier(size):
        """
        Generates majority predictions for the given category.
        inputs: 
            size: number of predictions to generate
        outputs: 
            array of majority predictions
        """
        return np.full(size, majority_value)
    
    return majority_classifier

Creating the baseline models for every category and saving them in a classifiers dictionary

In [33]:
classifiers = {}

categories = ['Openess to change', 'Self-enhancement', 'Conservation', 'Self-transcendence']

#create classifiers for each category and save them in the dictionary
for category in categories:
    #random uniform classifier
    random_uniform_name = f'random_uniform_classifier_{category}'
    classifiers[random_uniform_name] = create_random_uniform_classifier(category)

    #majority classifier
    majority_name = f'majority_classifier_{category}'
    classifiers[majority_name] = create_majority_classifier(category, majority_value=1) #da capire perchè majority_value=1

### BERT MODEL DEFINITION

In [129]:
#definition of the C_Model class for the first BERT-based model
class Bert_Model(torch.nn.Module):
    def __init__(self, model_name):
        super(C_Model, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained(
            model_name, 
            problem_type='multi_label_classification', 
            num_labels = 4, 
            return_dict=False)
        self.dropout = torch.nn.Dropout(p=0.3)
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, 4)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        #print(f"shape ids: {input_ids.shape}")
        #print(f"attention mask shape: {attention_mask.shape}")
        #print(f"token type ids shape: {token_type_ids.shape}")
        #print(f"Numero di valori restituiti: {len(outputs) if isinstance(outputs, tuple) else 1}")
        #print(f"Tipo outputs: {type(outputs)}")
        #print(f"tupla: {outputs}")
        #print("Output: ", outputs[0])
        #print(f"outputs logits: {outputs.logits}")
        outputs = self.dropout(outputs)
        #transposed_outputs = torch.transpose(outputs, 0, 1)
        #sequence_output = outputs.logits
        #print(f"Dimensione dell'output del modello BERT: {sequence_output.shape}")
        outputs = self.classifier(outputs)
        return outputs

'''
class roBERTa(torch.nn.Module):
    def __init__(self, model_name):
        super(roBERTa, self).__init__()
        self.roberta = AutoModel.from_pretrained(model_name, return_dict=False)
        self.dropout = torch.nn.Dropout(p=0.3)
        self.classifier = torch.nn.Linear(output_channels, 4)
        
    def forward(self, ids, mask, token_type_ids):
        _, output = self.roberta(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output = self.dropout(output)
        output = self.classifier(output)
        return output
'''

'\nclass roBERTa(torch.nn.Module):\n    def __init__(self, model_name):\n        super(roBERTa, self).__init__()\n        self.roberta = AutoModel.from_pretrained(model_name, return_dict=False)\n        self.dropout = torch.nn.Dropout(p=0.3)\n        self.classifier = torch.nn.Linear(output_channels, 4)\n        \n    def forward(self, ids, mask, token_type_ids):\n        _, output = self.roberta(ids, attention_mask=mask, token_type_ids=token_type_ids)\n        output = self.dropout(output)\n        output = self.classifier(output)\n        return output\n'

#### Bert Models

In [103]:
c_model = Bert_Model(model_name)
cp_model = Bert_Model(model_name)
cps_model = Bert_Model(model_name)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [31]:
print(c_model)

C_Model(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bia

## TASK 3 - METRICS

In [49]:
#defining per-category F1 score metric
def calculate_per_category_f1(y_true, y_pred):
    categories = ['Openess to change', 'Self-enhancement', 'Conservation', 'Self-transcendence']
    category_f1_scores = {}
    for category in categories:
        # Filter true and predicted labels for the current category
        category_indices = [i for i, cat in enumerate(y_true['category']) if cat == category]
        category_y_true = [y_true['Stance'][i] for i in category_indices]
        category_y_pred = [y_pred[i] for i in category_indices]
        
        # Calculate F1 score for the current category
        f1 = f1_score(category_y_true, category_y_pred, average='binary')
        category_f1_scores[category] = f1
    return category_f1_scores

#defining macro F1 score metric
def calculate_macro_f1(category_f1_scores):
    average_f1 = np.mean(list(category_f1_scores.values()))
    return average_f1

#defining the EvalPrediction object for Trainer
def calculate_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    labels = p.label_ids
    
    # F1 score per category
    category_f1_scores = calculate_per_category_f1(labels, preds)
    
    # F1 score macro
    macro_f1 = calculate_macro_f1(category_f1_scores)
    
    #results
    result = {
        'category_f1_scores': category_f1_scores,
        'macro_f1': macro_f1
    }
    
    return result

## TASK 4 - TRAINING AND EVALUATION

Training phase utils

In [89]:
#definition of the loss function
def loss(outputs, targets):
    return BCEWithLogitsLoss()(outputs, targets)

#definition of the optimizers
optimizer = Adam(c_model.parameters(), lr = 1e-5)

# Set seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

seeds = [42, 123, 2024]
#seeds = [456]

epochs = 10

Training function definitions

In [130]:
def trainBert(model, dataloader, optimizer, loss):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(dataloader, 0):
        ids = data['input_ids'].to(dtype=torch.long)
        mask = data['attention_mask'].to(dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(dtype=torch.long)
        labels = data['labels'].to(dtype=torch.float)
        optimizer.zero_grad()
        if model == cps_model:
            stance = data['stance'].to(dtype=torch.float)
            outputs = model(ids, mask, token_type_ids, stance)
        else:
            outputs = model(ids, mask, token_type_ids)
            
        loss = loss(outputs, labels)
        loss.backward()
        optimizer.step()
  
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(ids)
            print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

'''
def trainBert(model, dataloader):
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader)*epochs)
    size = len(dataloader.dataset)
    print(f"Size: {size}")
    model.train()
    for batch, data in enumerate(dataloader, 0):
        ids = data['ids'].to(device, dtype=torch.long)
        print(f"ids: {ids.shape}")
        mask = data['mask'].to(device, dtype=torch.long)
        print(f"mask: {mask.shape}")
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
        print(f"token_type_ids: {token_type_ids.shape}")
        targets = data['targets'].to(device, dtype=torch.float)
        print(f"targets: {targets.shape}")

        outputs = model(ids, mask, token_type_ids)
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(ids)
            print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
'''

'\ndef trainBert(model, dataloader):\n    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)\n    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader)*epochs)\n    size = len(dataloader.dataset)\n    print(f"Size: {size}")\n    model.train()\n    for batch, data in enumerate(dataloader, 0):\n        ids = data[\'ids\'].to(device, dtype=torch.long)\n        print(f"ids: {ids.shape}")\n        mask = data[\'mask\'].to(device, dtype=torch.long)\n        print(f"mask: {mask.shape}")\n        token_type_ids = data[\'token_type_ids\'].to(device, dtype=torch.long)\n        print(f"token_type_ids: {token_type_ids.shape}")\n        targets = data[\'targets\'].to(device, dtype=torch.float)\n        print(f"targets: {targets.shape}")\n\n        outputs = model(ids, mask, token_type_ids)\n        loss = loss_fn(outputs, targets)\n\n        optimizer.zero_grad()\n        loss.backward()\n        optimizer.step()\n      

In [104]:
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n-------------------------------")
    trainBert(c_model, train_dataloaderc, optimizer, loss)

Epoch 1
-------------------------------


/var/folders/m8/7w492cy56wj0kv_2fglfp_1m0000gn/T/ipykernel_30521/485821001.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


shape ids: torch.Size([16, 512])
attention mask shape: torch.Size([16, 512])
token type ids shape: torch.Size([16, 512])
Numero di valori restituiti: 1
Tipo outputs: <class 'tuple'>
tupla: (tensor([[ 0.2276, -0.3119, -0.1341, -0.2436],
        [ 0.2584, -0.2041,  0.0708,  0.0494],
        [ 0.0896, -0.5345, -0.0756, -0.4472],
        [-0.0820, -0.1567,  0.2536, -0.3381],
        [-0.1061, -0.5465, -0.0939, -0.3195],
        [-0.2700, -0.5105, -0.0469, -0.4586],
        [ 0.1083, -0.4611,  0.1415, -0.3111],
        [-0.1949, -0.7291,  0.3562, -0.5145],
        [-0.2003, -0.5631,  0.1054, -0.1898],
        [ 0.1008, -0.4967,  0.3645, -0.3020],
        [-0.1032, -0.6310, -0.0365, -0.3743],
        [ 0.0210, -0.3348,  0.0278, -0.2329],
        [ 0.1214, -0.2296,  0.0142, -0.2578],
        [ 0.0927, -0.5098,  0.0243, -0.3915],
        [ 0.0526, -0.3478, -0.0186, -0.2643],
        [-0.1420, -0.3296, -0.1070, -0.3779]], grad_fn=<AddmmBackward0>),)
Output:  tensor([[ 0.2276, -0.3119, -0.1341, 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x4 and 768x4)